In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt
from math import pi
from math import exp

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip")
test_df =  pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip")
result_df = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/sample_submission.csv.zip")
train_df["color"].replace({"clear": (1), "white": (2),"blood":(3),"green":(4),"blue":(5),"black": 6 }, inplace=True)
test_df["color"].replace({"clear": (1), "white": (2),"blood":(3),"green":(4),"blue":(5),"black": 6 }, inplace=True)

In [ ]:
monsterClasses = {'Ghoul':[],'Goblin':[],'Ghost':[]}
for idx,i in train_df.iterrows():
    monsterClasses[i.type].append(i.tolist())

In [ ]:
def calculateNormal(attribute):

    mean = np.mean(attribute)#sum(attribute)/float(len(attribute))
    stDeviation = np.std(attribute) #sum([(x-mean)**2 for x in attribute]) / float(len(attribute)-1) 
    #stDeviation = sqrt(stDeviation)
    return[mean,stDeviation,len(attribute)]


    

def getClassModel(monsterClass):
    monsterClass_df = pd.DataFrame(columns = train_df.columns,data = monsterClass)
    model = []
    for col in monsterClass_df.columns:
        if(col == 'id' or col == 'type' or col == 'color'): continue
        model.append(calculateNormal(monsterClass_df[col]))
    return model


    

In [ ]:
classModel = {'Ghoul':[],'Goblin':[],'Ghost':[]}
for monsterClassStr in monsterClasses:
    classModel[monsterClassStr]=(getClassModel(monsterClasses[monsterClassStr]))
print(classModel)



In [ ]:
def predictClass(testMonster):#for 1 monster
    probMonsterClasses = {'Ghoul':0,'Goblin':0,'Ghost':0}
    for monsterClass in classModel:
        probMonsterClasses[monsterClass] = 1/3 #(classModel[monsterClass][0][2]/(len(train_df)))
        #print(probMonsterClasses[monsterClass])
        for i in range(len(classModel[monsterClass])):
            mean = classModel[monsterClass][i][0]
            stdev = classModel[monsterClass][i][1]
            exponent = exp(-((testMonster[i+1]-mean)**2 / (2 * stdev**2 )))
           # print((1 / (sqrt(2 * pi) * stdev)))
            res =  (1 / (sqrt(2 * pi) * stdev)) * exponent
            probMonsterClasses[monsterClass] *= res
    return probMonsterClasses
                    

In [ ]:
def normalizePrediction(res):
    sumValues = sum(res.values())
    for i in res:
        res[i]=res[i]/sumValues
    return(res)



results = []
for index,monster in test_df.iterrows():
    #print(monster)
    results.append(predictClass(monster))

print(results[0:25])
print("Results after normalize:")
for i in range(len(results)):
    results[i] = normalizePrediction(results[i])
print(results[0:25])
    

In [ ]:
from sklearn.naive_bayes import MultinomialNB

train_df["type"].replace({"Ghoul": 0, "Goblin": 1,"Ghost":2}, inplace=True)
features = train_df.copy()
labels = features.pop('type')
features = features['color']
features = np.array(features)

test_features = test_df.copy()
test_features = test_features['color']
test_features = np.array(test_features)

classifier = MultinomialNB()

classifier.fit(features.reshape(-1, 1),labels)
color_probabilities = classifier.predict_proba(test_features.reshape(-1, 1))
print(color_probabilities)
#for i in range(len(results)):
    #results[i]["Ghoul"] *=  color_probabilities[i][0] 
    #results[i]["Goblin"] *= color_probabilities[i][1] 
    #results[i]["Ghost"] *=  color_probabilities[i][2]

print(results[0:25])
final_results =[]
for i in results:
    max_key = max(i, key=i.get)
    final_results.append(max_key)



In [ ]:
result_df.type = final_results
filename = "submission.csv"
result_df.to_csv(filename,index = False)